In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# r: (Kxd)
def log_likelihood(X,K,pi,mean,s):
    n, d = X.shape
    l = 0
    eps = 1e-5
    for i in range(n):
        ri = 0
        for k in range(K):
            product = np.prod(s[k])
            if product==0:
                product += eps
            val=pi[k]*np.exp(-0.5*np.sum((X[i]-mean[k])**2/s[k]))/np.sqrt(2*np.pi*product)
            ri += val
        if ri>0:
            l+=np.log(ri)
    return -l

# X: (nxd)
# K: number of clusters
# s: covariance matrix (Kxd) diagonal values
# mean: mean matrix (Kxd)
# pi: (1xK)
# r: (nxK)
def e_step(X,K,mean,s,pi):
    n, d = X.shape
    r = np.zeros((n,K))
    eps = 1e-5
    for i in range(n):
        ri = np.zeros((K,))
        for k in range(K):
            product = np.prod(s[k])
            if product==0:
                product += eps
            ri[k] = pi[k]*np.exp(-0.5*np.sum((X[i]-mean[k])**2/s[k]))*product**(-0.5)
        r[i] = ri
    
    ri = np.sum(r, axis=1)
    l = log_likelihood(X,K,pi,mean,s)
    for i in range(n):
        if ri[i]!=0:
            r[i]/=ri[i]
    return l,r

# X: input data points (nxd)
# K: number of clusters
def diagonal_gmm(X,K):
    max_iter = 500
    tol = 1e-5
    prev_l = 0
    n,d = X.shape
    # random initialization
    mean = np.random.uniform(tol,1,(K,d))
    s = np.random.uniform(tol,1,(K,d))
    pi = np.full((K), 1/K)
    for i in range(max_iter):
        # expectation step
        l,r = e_step(X,K,mean,s,pi)
        print("iter:", i, "   l:", round(l, 4))
        # check log likelihood
        if i>0 and np.abs(l-prev_l)<=tol*np.abs(l):
            break
        prev_l = l
        rk = np.sum(r, axis=0)
        # update mixing coeffs
        pi = rk/n
        # update mean
        for k in range(K):
            num = np.zeros((d))
            for i in range(n):
                num += r[i][k]*X[i]
            mean[k] = num/rk[k]
        # update covariance
        for k in range(K):
            num = np.zeros((d,))
            for i in range(n):
                num += r[i][k]*(X[i]-mean[k])**2
            s[k] = num/rk[k]
    return mean, s, pi, r, l


In [ ]:
X = np.loadtxt(open("gmm_dataset.csv", "rb"), delimiter=",")
ls = []
params = []
for k in range(1,11):
    print("k: ", k)
    mean, s, pi, r, l = diagonal_gmm(X,k)
    print("-----")
    ls.append(l)
    arr = [mean, s, pi, r]
    params.append(arr)

# output shows the log likelihood value in each iteration for each k value

In [ ]:
# plot k with negative log-likelihood
k_val = np.arange(1,11)
plt.plot(k_val, ls)
plt.title("Negative Log-Likelihood for Different Numbers of Clusters (k)")
plt.xlabel("k")
plt.ylabel("Negative Log-Likelihood")
plt.show()

In [ ]:
# optimal k value is 6
i = 5
mean = params[i][0]
var = params[i][1]
pi = params[i][2]
sorted_idx = np.argsort(pi)
print("k: ", i+1)
print("mean: ", mean[sorted_idx])
print("covariance: ", var[sorted_idx])
print("mixing coefficients: ", pi[sorted_idx])